<a href="https://colab.research.google.com/github/hanarayan/EPAM_PRACTICE/blob/main/PracticeEPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

In [3]:
from pyspark import SparkContext
import csv,json , re
from io import StringIO
# Check if a SparkContext already exists
try:
    sc = SparkContext.getOrCreate()
    print("Using existing SparkContext")
except ValueError:
    # If not, create a new one
    sc = SparkContext("local", "Linkdin Example")
    print("Created a new SparkContext")


Using existing SparkContext


In [4]:
rdd  = sc.textFile("/content/LinkedIn people profiles datasets.csv")
rdd_raw = rdd.map(lambda line: line.replace(',"', '\t"'))
print (rdd_raw.take(2))

['"timestamp"\t"id"\t"name"\t"city"\t"country_code"\t"region"\t"current_company:company_id"\t"current_company:name"\t"position"\t"following"\t"about"\t"posts"\t"groups"\t"current_company"\t"experience"\t"url"\t"people_also_viewed"\t"educations_details"\t"education"\t"avatar"\t"languages"\t"certifications"\t"recommendations"\t"recommendations_count"\t"volunteer_experience"\t"сourses"', '"2023-01-10"\t"catherinemcilkenny"\t"Catherine Fitzpatrick (McIlkenny), B.A"\t"Canada"\t"CA"\t"NA"\t"null"\t""\t"Snr Business Analyst at Emploi et Développement social Canada (EDSC) / Employment and Social Development Canada (ESDC)"\t"null"\t"null"\t"[{""attribution"":""Liked by Catherine Fitzpatrick (McIlkenny), B.A""\t""img"":""https://media.licdn.com/dms/image/C5622AQGesy-cKVfCaA/feedshare-shrink_2048_1536/0/1668547197397?e=2147483647&v=beta&t=hK5L_Ky-Ox4m90KT5u-roJB4C6BtWmD5UnOxYRvMucI""\t""link"":""https://www.linkedin.com/posts/esdcedsc_esw2022-activity-6998394190682341376-nMQh""\t""title"":""It’s 

In [5]:
header = rdd_raw.first()
rdd_no_header = rdd_raw.filter(lambda line: line != header)
print(f"Total rows (Including header): {rdd_raw.count()}")
print(f"Total rows (excluding header): {rdd_no_header.count()}")

Total rows (Including header): 1001
Total rows (excluding header): 1000


In [6]:


header_columns = header.split("\t")
header_columns = [col.strip('""').strip().lower() for col in header_columns]

print("Available Columns:")
for idx, column in enumerate(header_columns):
    print(f"Index: {idx}, Column: {column}")



Available Columns:
Index: 0, Column: timestamp
Index: 1, Column: id
Index: 2, Column: name
Index: 3, Column: city
Index: 4, Column: country_code
Index: 5, Column: region
Index: 6, Column: current_company:company_id
Index: 7, Column: current_company:name
Index: 8, Column: position
Index: 9, Column: following
Index: 10, Column: about
Index: 11, Column: posts
Index: 12, Column: groups
Index: 13, Column: current_company
Index: 14, Column: experience
Index: 15, Column: url
Index: 16, Column: people_also_viewed
Index: 17, Column: educations_details
Index: 18, Column: education
Index: 19, Column: avatar
Index: 20, Column: languages
Index: 21, Column: certifications
Index: 22, Column: recommendations
Index: 23, Column: recommendations_count
Index: 24, Column: volunteer_experience
Index: 25, Column: сourses


In [7]:
#rdd_split = rdd_no_header.map(lambda line: line.split(","))

def parse_csv_line(lines):
    reader = csv.reader(lines, delimiter='\t')  # Change delimiter if needed
    return [row for row in reader]

rdd_split = rdd_raw.filter(lambda row: row != header).mapPartitions(parse_csv_line)
print (rdd_split.take(1))

[['2023-01-10', 'catherinemcilkenny', 'Catherine Fitzpatrick (McIlkenny), B.A', 'Canada', 'CA', 'NA', 'null', '', 'Snr Business Analyst at Emploi et Développement social Canada (EDSC) / Employment and Social Development Canada (ESDC)', 'null', 'null', '[{"attribution":"Liked by Catherine Fitzpatrick (McIlkenny), B.A"\t"img":"https://media.licdn.com/dms/image/C5622AQGesy-cKVfCaA/feedshare-shrink_2048_1536/0/1668547197397?e=2147483647&v=beta&t=hK5L_Ky-Ox4m90KT5u-roJB4C6BtWmD5UnOxYRvMucI"\t"link":"https://www.linkedin.com/posts/esdcedsc_esw2022-activity-6998394190682341376-nMQh"\t"title":"It’s never too late to open an RESP. Find out how a little can go a long way for a child’s education and how saving can set them up for success!…"},{"attribution":"Liked by Catherine Fitzpatrick (McIlkenny), B.A"\t"img":"https://media.licdn.com/dms/image/C4E22AQHMWUhPL9AG5A/feedshare-shrink_800/0/1667668831230?e=2147483647&v=beta&t=LdI-tVu3yyWKpIm6MOsjYY0rsLMsLMjglOXvgjxPdyw"\t"link":"https://www.linkedi

In [8]:

def clean_column(code):
    cleaned_code = str(code).strip().strip('"')
    if not cleaned_code or cleaned_code.lower() in ["null", "no data","--"]:
        return "Not Available"
    return cleaned_code

In [9]:
def clean_digit(digit_value):
    try:
        return int(digit_value) if digit_value.isdigit() else 0
    except ValueError:
        return 0

In [10]:

country_code_index = header_columns.index("country_code")
print(country_code_index)
country_codes_rdd = rdd_split.map(lambda row: clean_column(row[country_code_index]))
distinct_countries = country_codes_rdd.distinct().sortBy(lambda x: x.lower()).collect()

print("Distinct Country Codes:")
for country in distinct_countries:
    print(country)

4
Distinct Country Codes:
AE
AM
AR
AT
AU
BA
BE
BG
BH
BN
BR
BS
CA
CD
CH
CI
CL
CN
CO
CR
CZ
DE
DK
DZ
EG
ES
FI
FR
GB
GE
GH
GM
GR
HK
ID
IE
IL
IN
IQ
IR
IT
JP
KR
KW
LK
LV
MA
MK
MO
MX
MY
NG
NL
NO
Not Available
NZ
OM
PE
PH
PK
PL
PT
QA
RO
RS
RU
SA
SD
SE
SG
SK
SN
SY
TH
TR
TW
UK
US
UY
VE
VN
ZA
ZM


In [11]:
region_index = header_columns.index("region")
regions_rdd  = rdd_split.map(lambda row: (clean_column(row[region_index]), 1))
region_counts = regions_rdd.reduceByKey(lambda a, b: a + b)
region_counts_result = region_counts.sortBy(lambda x: x).collect()

print("Region:" )
for region, count in region_counts_result:
    print(f"{region}, Count: {count}")


Region:
AF, Count: 26
AS, Count: 144
EU, Count: 209
NA, Count: 360
Not Available, Count: 184
OC, Count: 41
SA, Count: 36


In [12]:
company_name_index = header_columns.index("current_company:name")
company_names_rdd = rdd_split.map(lambda row: clean_column(row[company_name_index]))
distinct_company_names_rdd = company_names_rdd.distinct().sortBy(lambda x: x.lower()).collect()

print("Distinct company names:")
for company in distinct_company_names_rdd:
    print(company)

Distinct company names:
12th ave Thrift
3BlindMiceUSA/Costco/Graber
3M
A Local Hospital
A-Plan Insurance
A-reco Stockholm AB
A.S.C. ELEKTRİK MOTORLARI A.Ş.
ABC Therapy
ABN AMRO Bank N.V.
Absa Group
Abu Dhabi Commercial Bank
Abu Qir Petrolum
Academy of Holy Angels
Access Garage Doors
Accuspec Electronics, LLC
Aerotech Golf .LLC
African Reinsurance Brokers
African Spear
Afterburner
AGL Australia
Ailsa Sutcliffe Training Ltd.
Al Neem General Maintenance
Alcaldia de Cantagallo
ALDI Stores Australia
Alfardan Jewellery
Align Technology
All Personal, Inc
Allergan
Allianz Partners
Allied Dermatology
Alnylam Pharmaceuticals
Alshaya Group
Alten Sweden
Altour
Amarillo College
Amazon
Amdok
America dental
American Century Investments
American Heart Association
AMMSCPA
Analog Devices
ANAND AND ANAND
Animal Eye Guys
ANZ
Applied Materials
Aragen Life Sciences
ARAM Production House - آرام للإنتاج
Arbortech
Arc En ciel
ARCHIRODON CONSTRUCTION(OVERSEAS) COMPANY S.A. - DUBAI BRANCH
Ardea Partners
Arlingto

In [13]:
following_index = header_columns.index("following")
name_index = header_columns.index("name")

people_rdd = rdd_split.map(lambda row: (
    clean_digit(clean_column(row[following_index])),
    clean_column(row[name_index])
))


sorted_people_rdd = people_rdd.sortBy(lambda x: x[0], ascending=False)


top_10_followed = sorted_people_rdd.take(10)


print("Top 10 Most-Followed People:")
for following, name in top_10_followed:
    print(f"{name}: {following} followers")


Top 10 Most-Followed People:
Eloise Pamela Pusch: 617 followers
Hashan Rathnayaka: 515 followers
Giovanna Panarella: 500 followers
Jessica McCray: 500 followers
Andres Jaramillo Moyano: 500 followers
Jörgen Domby: 500 followers
Claire Sykes: 500 followers
Julia Garcia Antenucci: 500 followers
Prateek Chitpur: 500 followers
Christian Zerial: 500 followers


In [14]:
def clean_string(raw_str):
    """Cleans raw JSON string by fixing formatting issues."""
    cleaned_str = raw_str.strip()  # Remove leading/trailing spaces
    cleaned_str = cleaned_str.replace("\t", ",")  # Fix delimiters
    cleaned_str = cleaned_str.replace("['", "[")  # Fix opening bracket
    cleaned_str = cleaned_str.replace("']", "]")  # Fix closing bracket
    cleaned_str = cleaned_str.replace('""', 'null')  # Handle empty values correctly
    return cleaned_str

def safe_json_loads(raw_str):
    """Safely loads JSON string, handling decoding errors."""
    try:
        return json.loads(raw_str)
    except json.JSONDecodeError as e:
        print(f"JSON Parse Error: {raw_str} | Error: {e}")  # Debugging output
        return None  # Return None for invalid JSON rows

In [ ]:
post_index = header_columns.index("posts")
name_index = header_columns.index("name")


def extract_profile_likes(row):
    """Extracts profile name, falling back to 'name' column if attribution is missing."""
    post_list = safe_json_loads(clean_string(row[post_index]))

    if post_list:
        extracted_data = []
        for post in post_list:
            profile_name = post.get("attribution", row[name_index]).replace("Liked by ", "") if post.get("attribution") else row[name_index]
            extracted_data.append((profile_name, 1))
        return extracted_data
    return []

profile_likes_rdd = rdd_split.flatMap(extract_profile_likes)

likes_count_rdd = profile_likes_rdd.reduceByKey(lambda a, b: a + b)

all_profiles_rdd = rdd_split.map(lambda row: row[name_index]).distinct()

likes_count_dict = dict(likes_count_rdd.collect())

final_sorted_likes_count = sorted([(profile, likes_count_dict.get(profile, 0)) for profile in all_profiles_rdd.collect()], key=lambda x: x[0])

print("Total Posts Liked Per Profile (Sorted by Name, Including 0 Likes):")
for profile, count in final_sorted_likes_count:
    print(f"{profile}: {count}")

In [21]:
# Extract column indices
education_index = header_columns.index("education")
name_index = header_columns.index("name")  # Profile name for association

# Apply cleaning and JSON transformation
education_rdd = rdd_split.map(lambda row: safe_json_loads(clean_string(row[education_index]))).filter(lambda x: x is not None)

# Debugging output to verify JSON parsing
print("Cleaned JSON Output:", education_rdd.take(10))

def extract_education_profiles(row):
    """Extracts education institutions from 'title' tag within JSON, linked to profiles."""
    education_list = safe_json_loads(clean_string(row[education_index]))  # Convert 'education' column to JSON

    if education_list:  # Ensure JSON conversion is valid
        extracted_data = []
        for education in education_list:
            institution = education.get("title", "Unknown Institution")  # Get institution name
            extracted_data.append((institution, 1))  # Count occurrences
        return extracted_data
    return []  # Return empty list if no valid education data

# Create RDD mapping education institutions to profile counts
education_count_rdd = rdd_split.flatMap(extract_education_profiles)

# **Aggregate counts for each institution**
institution_counts_rdd = education_count_rdd.reduceByKey(lambda a, b: a + b)

# **Extract all unique education institutions to ensure missing entries are shown with `0` counts**
all_institutions_rdd = education_rdd.flatMap(lambda edu_list: [edu.get("title", "Unknown Institution") for edu in edu_list]).distinct()

# Convert counts to a dictionary for lookup
institution_count_dict = dict(institution_counts_rdd.collect())

# **Ensure missing institutions appear with a count of `0`**
final_sorted_institutions = sorted([(institution, institution_count_dict.get(institution, 0)) for institution in all_institutions_rdd.collect()], key=lambda x: x[0].lower())

# **Print final sorted results**
print("Total Profiles by Education Institution (Sorted Alphabetically):")
for institution, count in final_sorted_institutions:
    print(f"{institution}: {count}")

Cleaned JSON Output: [[{'degree': 'Bachelor of Arts (B.A.) Honours', 'end_year': '2011', 'field': 'English Language and Literature', 'meta': '2009 - 2011', 'start_year': '2009', 'title': "Queen's University Belfast", 'url': "https://www.linkedin.com/school/queen's-university-belfast/"}, {'degree': 'Business Analysis Essentials Certification', 'end_year': '2021', 'field': 'Business Analysis', 'meta': '2019 - 2021 The Business Analysis Essentials program delivers practical, hands-on, experience-based training in the core knowledge and skills of business analysis. Courses: Fundamentals of Business AnalysisAccelerated Requirements, Elicitation and Analysis Facilitation Techniques for Business AnalystsModelling Business Processes and Workflows Use Case Modelling Validating and Testing Requirements Business Analysis in an Agile Environment', 'start_year': '2019', 'title': 'Algonquin College of Applied Arts and Technology', 'url': 'https://www.linkedin.com/school/algonquincollege/'}, {'degree